In [15]:
import pandas as pd
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ✅ 로그인 쿠키 주입 함수
def set_cookies(driver, cookies):
    for cookie in cookies:
        driver.add_cookie(cookie)

# ✅ 작가 홈 방문 후 작품 수 크롤링
def get_total_works(driver, author_url):
    driver.get(author_url)
    time.sleep(1)
    if "community/u/" in author_url:
        try:
            work_elements = driver.find_elements(By.CSS_SELECTOR, 'li._1w7ywmw0')
            return len(work_elements)
        except Exception as e:
            print(f"작가 홈 크롤링 오류: {e}")
            return 0
    elif "artistTitle" in author_url:
        try:
            total_text = driver.find_element(By.CSS_SELECTOR, 'span.ComponentHead__count--bRLNZ').text.strip()
            work_count = int(re.search(r'\d+', total_text).group())
            return work_count
        except Exception as e:
            print(f"artistTitle 페이지 크롤링 오류: {e}")
            return 0
    else:
        print("알 수 없는 작가 페이지 형태")
        return 0

# ✅ 작품 페이지에서 작가 정보 및 작품 수 크롤링
def crawl_author_works(title_id, webtoon_url, driver):
    driver.get(webtoon_url)
    time.sleep(1)

    # 🚀 쿠키를 세션에 주입하고 refresh
    driver.refresh()
    time.sleep(1)

    # ✅ 각 변수 초기화
    writer_works = 0
    illustrator_works = 0
    writer_illustrator_works = 0
    original_writer_works = 0
    total_works = 0
    is_original_exist = 0

    # ✅ 작가 영역 개수만큼 루프 돌리기
    author_blocks = driver.find_elements(By.CSS_SELECTOR, 'span.ContentMetaInfo__category--WwrCp')
    for i in range(len(author_blocks)):
        try:
            # 🚀 반드시 fresh 요소를 다시 가져오기
            author_blocks = driver.find_elements(By.CSS_SELECTOR, 'span.ContentMetaInfo__category--WwrCp')
            block = author_blocks[i]

            role_text = block.text.strip()
            author_name = block.find_element(By.CSS_SELECTOR, 'a').text.strip()
            author_link = block.find_element(By.CSS_SELECTOR, 'a').get_attribute("href")
            print(f"[{title_id}] {role_text} | {author_name} | {author_link}")

            works_count = get_total_works(driver, author_link)

            if "글/그림" in role_text:
                writer_illustrator_works = works_count
            elif "글" in role_text:
                writer_works = works_count
            elif "그림" in role_text:
                illustrator_works = works_count
            elif "원작" in role_text:
                original_writer_works = works_count
                is_original_exist = 1

            total_works += works_count

        except Exception as e:
            print(f"작가 정보 수집 오류: {e}")
            continue

    # ✅ 결과 출력
    print(f"✅ [title_id: {title_id}] 글: {writer_works}, 그림: {illustrator_works}, 글/그림: {writer_illustrator_works}, 원작: {original_writer_works}, 총합: {total_works}, 원작여부: {is_original_exist}")

    return {
        'title_id': title_id,
        'writer_works': writer_works,
        'illustrator_works': illustrator_works,
        'writer_illustrator_works': writer_illustrator_works,
        'original_writer_works': original_writer_works,
        'total_works': total_works,
        'is_original': is_original_exist
    }

# ✅ 🔥 크롤링 실행부
if __name__ == "__main__":
    # 🔥 로그인 쿠키 준비
    cookies = [
        {"name": "NID_AUT", "value": "kY4ygf1UDrtliANsS5KDqCPwtEfDHpwGd9zwoYYhifJUl0bwDDDggqXtfCxU/Urv", "domain": ".naver.com"},
        {"name": "NID_SES", "value": "AAAB6BGACbhtslHQ2Mn/TVA3rT5U74dPcjz6gOenwIMfKPRALs4JGmJ6nxxmtUGdRizKpXEeX/13rA7Oxsnjg1xqAcQZBUCTCNn/mHW7k+hJd6Rcljg0utyz8e/9BmLRRPIccLmb0MUiy6MopWTBJCxpF1f3IW1zgoXUao+jFlGxkCIKI+LdV3KWeYBshfNtpxHx2K+ADZFVZUpkUD/b+rYuYfQwqF0N6zR0vUJON/X2pPmi5gkw9uj9NVqSqjwGyPtOYNB4T/Oi4FWYIMU6MEOeZK2qCO5kXZTjswluXsJuLhiTXBEBlOzRB5ZwPZVBTAdZ2kdg5iHkyhKXTnJrS7EhJloLJXuNhrY8B5GEOJvwohgyqBaSCQxg+9diIrFu96nqZMivUTeDQ+cNejVZp8x1Jhoq4bWmcZ3x+F9QW1c0xSTLNKuKpGpPYeOj7AQ9j1YD1OLJZD4sId/mU67sYyUjG4udvZkl7RSMA+NYsmuCzNskyE2FPOIVJSqrcSQ7Tnn1PULBFVGQwTDdknwoTE1jzOXCqL3ai6Kw67+eUUMwIATjupWzBhDm8Kdu9QXKcC6DQ+KuAxTcOqpyVF2PZTcrcK8XZp6y7oE6USMS7VMFI5CoNsgMT3odsowpwlx2n0ez7aX3pfOYGHlBYrBILIjLvpc=", "domain": ".naver.com"}
    ]

    # 🔥 CSV 읽기
    df = pd.read_csv('naver_작가 작품 수, 원작 여부 병합 전_only19.csv', encoding='utf-8')

    # 🔥 결과 저장용
    results = []

    # 🔥 폴더 생성
    folder_name = "19세_작가_작품수_결과"
    os.makedirs(folder_name, exist_ok=True)

    # 🔥 Selenium driver 실행
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # ✅ 쿠키 주입
    driver.get("https://comic.naver.com/")
    time.sleep(1)
    set_cookies(driver, cookies)
    driver.refresh()
    time.sleep(1)

    # ✅ 각 작품별 크롤링
    for idx, row in df.iterrows():
        title_id = row['title_id']
        webtoon_url = f"https://comic.naver.com/webtoon/list?titleId={title_id}"
        result = crawl_author_works(title_id, webtoon_url, driver)
        results.append(result)

    driver.quit()

    # 🔥 결과 CSV 저장
    df_result = pd.DataFrame(results)
    result_file = os.path.join(folder_name, "19세_작가_작품수_최종.csv")
    df_result.to_csv(result_file, index=False, encoding='utf-8-sig')
    print(f"🎉 최종 결과 저장 완료: {result_file}")


[697654] 황준호
∙
글/그림 | 황준호 | https://comic.naver.com/community/u/hwangjunho?lastPage=EpisodeList
✅ [title_id: 697654] 글: 0, 그림: 0, 글/그림: 10, 원작: 0, 총합: 10, 원작여부: 0
[628997] 김칸비
∙
글 | 김칸비 | https://comic.naver.com/community/u/carnbykim?lastPage=EpisodeList
작가 정보 수집 오류: list index out of range
✅ [title_id: 628997] 글: 9, 그림: 0, 글/그림: 0, 원작: 0, 총합: 9, 원작여부: 0
[700858] 팀 겟네임
∙
글 | 팀 겟네임 | https://comic.naver.com/artistTitle?id=103
[700858] 그림
설레임 | 설레임 | https://comic.naver.com/artistTitle?id=312439
[700858] 글
팀 겟네임
∙
아루아니 | 팀 겟네임 | https://comic.naver.com/artistTitle?id=103
✅ [title_id: 700858] 글: 5, 그림: 2, 글/그림: 0, 원작: 0, 총합: 12, 원작여부: 0
[821589] 지공
∙
글/그림 | 지공 | https://comic.naver.com/community/u/_jbwx9?lastPage=EpisodeList
✅ [title_id: 821589] 글: 0, 그림: 0, 글/그림: 2, 원작: 0, 총합: 2, 원작여부: 0
[703851] 공현곤
∙
글/그림 | 공현곤 | https://comic.naver.com/community/u/konggon?lastPage=EpisodeList
✅ [title_id: 703851] 글: 0, 그림: 0, 글/그림: 6, 원작: 0, 총합: 6, 원작여부: 0
[696617] 연제원
∙
글/그림 | 연제원 | https://comic.nav